### Notebook to Analyse and Plot Simulation Results

The code is adapted from [sumo-rl](https://github.com/LucasAlegre/sumo-rl/blob/main/outputs/plot.py).

#### Imports

In [2]:
import argparse
import glob
from itertools import cycle
import os

import matplotlib.pyplot as plt
plt.rcParams.update(plt.rcParamsDefault)
import numpy as np
import pandas as pd
import seaborn as sns

#### Plot Settings

In [3]:
sns.set(
    style="darkgrid",
    rc={
        "figure.figsize": (7.2, 4.45),
        #"text.usetex": True,
        "xtick.labelsize": 16,
        "ytick.labelsize": 16,
        "font.size": 15,
        "figure.autolayout": True,
        "axes.titlesize": 16,
        "axes.labelsize": 17,
        "lines.linewidth": 2,
        "lines.markersize": 6,
        "legend.fontsize": 15,
    },
)
colors = sns.color_palette("colorblind", 4)
# colors = sns.color_palette("Set1", 2)
# colors = ['#FF4500','#e31a1c','#329932', 'b', 'b', '#6a3d9a','#fb9a99']
dashes_styles = cycle(["-", "-.", "--", ":"])
sns.set_palette(colors)
colors = cycle(colors)

In [4]:
# functions

def moving_average(interval, window_size):
    if window_size == 1:
        return interval
    window = np.ones(int(window_size)) / float(window_size)
    return np.convolve(interval, window, "same")


def plot_df(df, color, xaxis, yaxis, ma=1, label=""):
    df[yaxis] = pd.to_numeric(df[yaxis], errors="coerce")  # convert NaN string to NaN value

    mean = df.groupby(xaxis).mean()[yaxis]
    std = df.groupby(xaxis).std()[yaxis]
    if ma > 1:
        mean = moving_average(mean, ma)
        std = moving_average(std, ma)

    x = df.groupby(xaxis)[xaxis].mean().keys().values
    plt.plot(x, mean, label=label, color=color, linestyle=next(dashes_styles))
    plt.fill_between(x, mean + std, mean - std, alpha=0.25, color=color, rasterized=True)

    # plt.ylim([0,200])
    # plt.xlim([40000, 70000])


In [6]:

prs = argparse.ArgumentParser(
    formatter_class=argparse.ArgumentDefaultsHelpFormatter, description="""Plot Traffic Signal Metrics"""
)
prs.add_argument("-f", nargs="+", default="../src/data/model_outputs/", required=False, help="Measures files\n")
prs.add_argument("-l", nargs="+", default=None, help="File's legends\n")
prs.add_argument("-t", type=str, default="", help="Plot title\n")
prs.add_argument("-yaxis", type=str, default="system_total_waiting_time", help="The column to plot.\n")
prs.add_argument("-xaxis", type=str, default="step", help="The x axis.\n")
prs.add_argument("-ma", type=int, default=1, help="Moving Average Window.\n")
prs.add_argument("-sep", type=str, default=",", help="Values separator on file.\n")
prs.add_argument("-xlabel", type=str, default="Time step (seconds)", help="X axis label.\n")
prs.add_argument("-ylabel", type=str, default="Total waiting time (s)", help="Y axis label.\n")
prs.add_argument("-output", type=str, default="sumo-rl/outputs/plot_MA_PPO", help="PDF output filename.\n")

args = prs.parse_args()
labels = cycle(args.l) if args.l is not None else cycle([str(i) for i in range(len(args.f))])

plt.figure()

# JH: new line of code
# main_df = pd.read_csv('sumo-rl/outputs/2way-single-intersection/dqn_conn0_ep2.csv')
# main_df = main_df[['step', 'system_total_waiting_time']]

#File reading and grouping
# for file in os.dir(args.f):
#     main_df = pd.DataFrame()
#     for f in glob.glob(file + "*"):
#         df = pd.read_csv(f, sep=args.sep)
#         if main_df.empty:
#             main_df = df
#         else:
#             main_df = pd.concat((main_df, df))



usage: ipykernel_launcher.py [-h] [-f F [F ...]] [-l L [L ...]] [-t T]
                             [-yaxis YAXIS] [-xaxis XAXIS] [-ma MA] [-sep SEP]
                             [-xlabel XLABEL] [-ylabel YLABEL]
                             [-output OUTPUT]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"260a1c88-6676-4c00-b9e1-50b77abe4b73" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/jenniferhahn/Library/Jupyter/runtime/kernel-v2-50194DgU2QoLCCvlJ.json


SystemExit: 2

/Users/jenniferhahn/Documents/GitHub/sumovenv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
for file in os.listdir(args.f):
    if file.startswith('waitingTime'):
        main_df = pd.DataFrame()
        main_df = pd.read_csv(args.f + file)
        main_df = main_df[[args.xaxis, args.yaxis]]

        # Plot DataFrame
        plot_df(main_df, xaxis=args.xaxis, yaxis=args.yaxis, label=next(labels), color=next(colors), ma=args.ma)

plt.title(args.t)
plt.ylabel(args.ylabel)
plt.xlabel(args.xlabel)
plt.ylim(bottom=0)

if args.output is not None:
    plt.savefig(args.output + ".pdf", bbox_inches="tight")

plt.show()

NameError: name 'args' is not defined